In [ ]:
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import tensorflow as tf
import numpy as np
from pydub import AudioSegment
from spleeter.separator import Separator
import os


MODEL_PATH = "final_model_singer.keras"
model = tf.keras.models.load_model(MODEL_PATH)


def singer_command(update: Update, context: CallbackContext) -> None:
    update.message.reply_text("Send a song 🎶, and I will extract vocals to recognize the singer!")


def handle_audio(update: Update, context: CallbackContext) -> None:
    voice = update.message.voice or update.message.audio
    if not voice:
        update.message.reply_text("Please send a valid audio file.")
        return

   
    audio_file = voice.get_file()
    audio_path = "song.ogg"
    audio_file.download(audio_path)

    
    audio = AudioSegment.from_file(audio_path)
    audio = audio.set_channels(1).set_frame_rate(48000)
    processed_audio_path = "processed_song.wav"
    audio.export(processed_audio_path, format="wav")

    
    vocals_dir = "spleeter_output"
    os.makedirs(vocals_dir, exist_ok=True)
    
    separator = Separator("spleeter:2stems")
    separator.separate_to_file(processed_audio_path, vocals_dir)

    
    file_name = os.path.splitext(os.path.basename(audio_path))[0]
    vocal_file_path = os.path.join(vocals_dir, file_name, "vocals.wav")

    
    audio = AudioSegment.from_file(vocal_file_path)
    audio = audio.set_channels(1).set_frame_rate(48000).set_sample_width(2)
    final_audio_path = "processed_vocals.wav"
    audio.export(final_audio_path, format="wav")

   
    audio_binary = tf.io.read_file(final_audio_path)
    audio_tensor, _ = tf.audio.decode_wav(audio_binary, desired_channels=1, desired_samples=48000)

    
    audio_tensor = tf.expand_dims(audio_tensor, axis=0)

   
    try:
        prediction = model.predict(audio_tensor)
        predicted_class = np.argmax(prediction, axis=1)[0]
        update.message.reply_text(f"🎤 Recognized singer: {predicted_class}")
    except Exception as e:
        update.message.reply_text(f"Error during prediction: {str(e)}")

def main():
    updater = Updater("TOKEN", use_context=True)
    dispatcher = updater.dispatcher

   
    dispatcher.add_handler(CommandHandler("singer", singer_command))

    
    dispatcher.add_handler(MessageHandler(Filters.voice | Filters.audio, handle_audio))

    updater.start_polling()
    updater.idle()

if __name__ == "__main__":
    main()
